## TODO

- manual weighting of negative and positive
- include b_words in negative
- speed comparison for word look vs vector

In [1]:
# quick hack for adding codenames package to the path
import sys
sys.path.append("../")

In [2]:
import os
from itertools import combinations

from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors

import numpy as np

In [3]:
MODELS_DIR = "../models"
# dev model with 1000 items
MODEL_FPATH = os.path.join(MODELS_DIR, "dev-wiki-news-300d-1M-subword.vec")

N_TEAM_WORDS = 10 # word for each team
N_ASSASSIN = 1 # n assassin carads
MIN_COMBO = 1 # min no cards to try and guess in one turn
MAX_COMBO = 4 # max no cards to try to guess in one turn

load the model

In [4]:
%%time
model = KeyedVectors.load_word2vec_format(MODEL_FPATH)

CPU times: user 22 s, sys: 158 ms, total: 22.2 s
Wall time: 22.1 s


Choose a set of words to represent the words on the board. Assuming all board words are in the vocab.

In [5]:
# simplistic filter to try and get non punctuation and more interesting words
filtered_vocab = [word for word in model.vocab.keys() if len(word) > 3]
n_total_words = (N_TEAM_WORDS*2) + N_ASSASSIN
board_words = np.random.choice(filtered_vocab, size=n_total_words, replace=False)
team_a_words = board_words[:N_TEAM_WORDS].tolist()
team_b_words = board_words[N_TEAM_WORDS:N_TEAM_WORDS*2].tolist()
assassin_word = board_words[-1].tolist()

In [6]:
team_a_words, team_b_words, assassin_word

(['severed',
  'Otto4711',
  'Dynegy',
  'Latest',
  'Hump',
  'horizontally',
  'Sourcing',
  'Jebel',
  'Angola',
  'Leichhardt'],
 ['Whitman',
  'statewide',
  'kitsch',
  'Ludvig',
  'Giraffe',
  'Polar',
  'ironclad',
  'delights',
  'Preventive',
  'Jigsaw'],
 'Dern')

In [13]:
def get_clue(kv, a_words, assassin, min_combo, max_combo):

    # all combos across different guess amounts
    combos = [item for i in range(min_combo,max_combo+1) for item in combinations(a_words, i)]

    clues = list()
    similarities = np.zeros(len(combos))

    for i, combo in enumerate(combos):
        similar = kv.most_similar(positive=combo, negative=[assassin])
        assert len(similar) > 0
        clue, similarity = similar[0]
        clues.append(clue)
        similarities[i] = similarity

    return clues[ind], similarities[ind], combos[ind]

In [16]:
clue, similarity, words = get_clue(kv=model,
                                   a_words=team_a_words,
                                   assassin=assassin_word,
                                   min_combo=MIN_COMBO,
                                   max_combo=MAX_COMBO)

In [17]:
clue, similarity, words

('vertically', 0.7011539340019226, ('severed', 'horizontally'))